In [ ]:
using Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

In [ ]:
using JSON
using Dates
using TimeZones
using Chain
using ResultTypes

using Base: @kwdef

In [ ]:
using SFAPI

In [ ]:
client = SFAPI.Token.Client(joinpath(@__DIR__, "credentials"))
tc = SFAPI.Token.fetch(client) |> unwrap

In [ ]:
tc = SFAPI.Token.refresh(tc)
account = SFAPI.Account.User(SFAPI.Query.get("account", tc.token))

In [ ]:
home_path = "/global/homes/$(account.name[1])/$(account.name)"
scratch_path = "/pscratch/sd/$(account.name[1])/$(account.name)"

In [ ]:
tc = SFAPI.Token.refresh(tc)
x = SFAPI.Query.get(
    "compute/jobs/perlmutter", tc.token;
    parameters=Dict(
        "index" => "0",
        "sacct" => "false",
        "kwargs" => "user=blaschke"
    )
) |> unwrap

In [ ]:
job_script = """#!/bin/bash
#SBATCH -q regular
#SBATCH -A nstaff
#SBATCH -N 1
#SBATCH -C cpu
#SBATCH -t 00:05:00
#SBATCH -J sfapi-test
#SBATCH --output=$(scratch_path)/sfapi/test.out
#SBATCH --error=$(scratch_path)/sfapi/test.error

echo "hi"
"""

job_script_file = "$(scratch_path)/sfapi/test.sh"

println(job_script)

In [ ]:
job_script_file

In [ ]:
tc = SFAPI.Token.refresh(tc)
cmd = SFAPI.Executable.run( 
    "cat > $(job_script_file) << EOF\n$(job_script)EOF",
    tc
)

t = SFAPI.Executable.result(cmd, tc)
while ! istaskdone(t)
    println("Wating for result ...")
    sleep(1)
end
println(fetch(t))

In [ ]:
tc = SFAPI.Token.refresh(tc)
cmd = SFAPI.Executable.run(
    "ls $(job_script_file)",
    tc
)

t = SFAPI.Executable.result(cmd, tc)
while ! istaskdone(t)
    println("Wating for result ...")
    sleep(1)
end
r = fetch(t)

In [ ]:
strip(r.output) == job_script_file